# **Finetuning ibm granite instruct 7b using qlora for wealthwiser**

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Check if Colab is using a T4 GPU
!nvidia-smi

Wed Oct 23 17:18:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install transformers datasets accelerate bitsandbytes loralib peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00


In [ ]:
# Install necessary libraries if not already installed
#!pip install transformers peft bitsandbytes accelerate

import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer

# Load the IBM Granite 7b Instruct model and tokenizer from Hugging Face
model_name = "ibm-granite/granite-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the padding token of the tokenizer to the EOS token.
tokenizer.pad_token = tokenizer.eos_token

# BitsAndBytesConfig for quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Quantize to 4-bit precision
    bnb_4bit_quant_type="nf4",  # NF4 quantization type
    bnb_4bit_compute_dtype=torch.float16,  # Model computed in float16 precision
)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Disable the caching mechanism of the Transformers model.
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# Define LoRA configuration
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
# Create PeftModel
model = get_peft_model(model, peft_config)

# Print trainable parameters
model.print_trainable_parameters()

trainable params: 33,554,432 || all params: 6,772,035,584 || trainable%: 0.4955


In [ ]:
# Load your dataset (CSV file)
data = pd.read_csv("/content/Testing dataset - GPT.csv")  # Replace with your dataset file
# Ensure to filter to only the 'question' and 'answer' columns if applicable
data = data[['question', 'answer']]

# Prepare dataset for training
train_data = [{'text': f"{row['question']} {tokenizer.eos_token} {row['answer']}"} for index, row in data.iterrows()]
dataset = Dataset.from_list(train_data)

# Define training arguments
output_dir = "/content/drive/MyDrive/granite_finetuned/results"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,  # Adjust based on your memory
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=500,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

# Set max sequence length
max_seq_length = 512

# Create SFTTrainer instance
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2009 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Ensure normalization layers are in float32
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
# Start training
trainer.train()

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("/content/drive/MyDrive/granite_finetuned/outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('/content/drive/MyDrive/granite_finetuned/outputs')
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.10/dist-packages/peft/mapping.py:172: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'ibm-granite/granite-7b-instruct' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [ ]:
text = "###Human: Give me some good reasons to buy the copyrighter web app. ### Assistant: "
device = "cuda:0"

# Tokenize a text input and convert it to a PyTorch tensor on the specified device.
inputs = tokenizer(text, return_tensors="pt").to(device)

In [ ]:
model.print_trainable_parameters()

trainable params: 0 || all params: 6,772,035,584 || trainable%: 0.0000


# **Inference**

# **Old Code**

In [ ]:
import torch

# Ensure the model is in evaluation mode, moved to the correct device, and set to half precision (if needed)
model.eval().to(device)

# Optionally set the model to half precision (this only applies to model weights, not input tokens)
model.half()  # This ensures the model uses Half precision (16-bit float)

# Tokenize input text and convert to tensors
text = "###Human: How can I improve my position at work? ### Assistant: "
inputs = tokenizer(text, return_tensors="pt").to(device)

# Ensure that input_ids remain as Long type (do NOT convert to half)
# input_ids should NOT be converted to half precision, they must remain as Long tensors
# inputs['input_ids'] = inputs['input_ids'].half()  # Do NOT do this

# Generate text with no gradient calculations
with torch.no_grad():  # Disable gradient computation for inference
    outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=100)  # Ensure to use 'input_ids' correctly

# Decode and print the output
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


###Human: How can I improve my position at work? ### Assistant: 

Improving your position at work can be achieved through various strategies. Here are some suggestions to help you advance in your career:

1. **Set Clear Goals:** Establish specific, measurable, achievable, relevant, and time-bound (SMART) goals for your professional growth. This will enable you to focus your efforts and track your progress effectively.

2. **Develop Strong Skills:** Continuously work on impro


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/

In [ ]:
import gradio as gr
import torch

# Assume your model and tokenizer are loaded here
# For example, let's say you have loaded your model as `model` and tokenizer as `tokenizer`

device = "cuda" if torch.cuda.is_available() else "cpu"
model.eval().to(device)  # Ensure model is on the correct device
model.half()  # Use half precision for faster inference on GPU

# Function to handle user inputs and return model outputs
def chatbot_response(user_input):
    # Prepare the input for the model
    inputs = tokenizer(f"###Human: {user_input} ### Assistant:", return_tensors="pt").to(device)

    # Generate text with no gradient calculations
    with torch.no_grad():
        outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=512)

    # Decode the output and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Gradio Interface
iface = gr.Interface(
    fn=chatbot_response,           # Function to call when user inputs text
    inputs="text",                 # Input component (a text box)
    outputs="text",                # Output component (a text display)
    title="Wealth Wiser",            # Title of the interface
    description="Ask me anything!" # Description
)

# Launch the Gradio interface with public URL enabled
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ac89db8ae7de0db77f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **Full Working with Gradio**

In [ ]:
!pip install chromadb bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:0

In [ ]:
import requests
from bs4 import BeautifulSoup
import uuid
import chromadb
import re
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Initialize ChromaDB client and create/get the collection
client = chromadb.PersistentClient(path="vectorstore")
collection = client.get_or_create_collection(name="portfolio")

# Load your fine-tuned LLaMA 2 model and tokenizer from Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make sure the correct path is added based on your Colab setup (e.g., "/content/model_path")
#tokenizer = AutoTokenizer.from_pretrained("/content/finetuned_llama2")
#model = AutoModelForCausalLM.from_pretrained("/content/finetuned_llama2", torch_dtype=torch.float16).to(device)
model.eval().to(device)  # Ensure model is on the correct device
model.half()

# Function to fetch and process the data (same as in your current scraper)
def fetch_data():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    r = requests.get('https://www.5paisa.com/share-market-today/stocks-to-buy-or-sell-today', headers=headers)

    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('table', {'id': 'stock-table'})

        headers = [header.text.strip() for header in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:
            cols = [col.text.strip() for col in row.find_all('td')]
            rows.append(cols)

        data = []
        for row in rows:
            stock_info = dict(zip(headers, row))
            stock_info["TRADE PRICE"] = convert_to_type(stock_info.get("TRADE PRICE", "0"))
            stock_info["SL"] = convert_to_type(stock_info.get("SL", "0"))
            stock_info["TARGET 1"] = convert_to_type(stock_info.get("TARGET 1", "0"))
            stock_info["TARGET 2"] = convert_to_type(stock_info.get("TARGET 2", "0"))
            trade_price = stock_info["TRADE PRICE"]
            target_prices = [stock_info.get("TARGET 1", 0), stock_info.get("TARGET 2", 0)]
            risk_level = calculate_risk(trade_price, target_prices)
            stock_info["RISK"] = risk_level
            data.append(stock_info)

        return data
    else:
        print(f"Failed to fetch page. Status code: {r.status_code}")
        return []

# Define the risk category function
def calculate_risk(trade_price, target_prices):
    trade_price = float(trade_price)
    risk_levels = []

    for target_price in target_prices:
        target_price = float(target_price)
        percentage_difference = ((target_price - trade_price) / trade_price) * 100

        if percentage_difference < 5:
            risk_levels.append('low')
        elif 5 <= percentage_difference < 10:
            risk_levels.append('mid')
        else:
            risk_levels.append('high')

    return max(risk_levels)

# Function to convert string to the appropriate type
def convert_to_type(value):
    try:
        if '.' in value:
            return float(value)
        return int(value)
    except ValueError:
        return value

# Insert or update data (same as in your current logic)
def insert_update_data(data):
    for entry in data:
        stock_name = entry["STOCK"]
        if delete_existing_stock(stock_name):
            print(f"Existing stock '{stock_name}' found and deleted. Updating with new values...")

        collection.add(
            documents=[stock_name],
            metadatas={
                "action": entry["ACTION"],
                "trade_price": entry["TRADE PRICE"],
                "sl": entry["SL"],
                "target_1": entry["TARGET 1"],
                "target_2": entry["TARGET 2"],
                "risk": entry["RISK"]
            },
            ids=[str(uuid.uuid4())]
        )

# Delete existing stock data
def delete_existing_stock(stock_name):
    results = collection.get()
    stock_ids_to_delete = []
    for doc_id, doc, meta in zip(results['ids'], results['documents'], results['metadatas']):
        if meta.get('action') == stock_name:
            stock_ids_to_delete.append(doc_id)

    if stock_ids_to_delete:
        collection.delete(ids=stock_ids_to_delete)
        return True
    return False

# Function to generate trading advice
def generate_trading_advice(user_query):
    # Tokenize input text and convert to tensors
    input_text = f"###Human: {user_query} ### Assistant: "
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Ensure that input_ids remain as Long type
    with torch.no_grad():  # Disable gradient computation for inference
        outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=100)

    # Decode and process the model's response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Now, extract profit target from the user's request using regex
    profit_match = re.search(r'(\d+)% profit', user_query)
    if profit_match:
        profit_percentage = int(profit_match.group(1))
    else:
        profit_percentage = 5  # Default to 5% if not provided

    # Retrieve data from vector DB that matches the user's goal
    matching_stocks = []
    results = collection.get()
    for doc, meta in zip(results['documents'], results['metadatas']):
        target_1 = float(meta["target_1"])
        trade_price = float(meta["trade_price"])
        potential_profit = ((target_1 - trade_price) / trade_price) * 100

        if potential_profit >= profit_percentage:
            matching_stocks.append({
                "stock": doc,
                "trade_price": trade_price,
                "target_1": target_1,
                "risk": meta["risk"],
                "potential_profit": potential_profit
            })

    # Construct the detailed response message for the user
    if matching_stocks:
        advice = f"I found some stocks that may help you achieve a {profit_percentage}% profit:\n\n"

        for stock in matching_stocks:
            # Detail breakdown for each stock
            stock_details = (
                f"**Stock: {stock['stock']}**\n"
                f" - Trade Price: {stock['trade_price']}\n"
                f" - Target Price: {stock['target_1']}\n"
                f" - Potential Profit: {stock['potential_profit']:.2f}%\n"
                f" - Risk Level: {stock['risk'].capitalize()}\n\n"
                f"**Analysis**: If you buy {stock['stock']} at {stock['trade_price']}, you could potentially make a {stock['potential_profit']:.2f}% profit when the price reaches {stock['target_1']}. However, the risk is {stock['risk'].capitalize()}, which indicates {provide_risk_explanation(stock['risk'])}.\n\n"
            )
            advice += stock_details

        # Offer alternative choices based on higher or lower risks/profits
        advice += "**Other choices to consider based on your risk tolerance and profit goals:**\n"
        for stock in matching_stocks:
            # Provide some alternative stocks with a bit more or less profit/risk
            alternative = (
                f"- {stock['stock']}: Trade at {stock['trade_price']}, "
                f"Potential Profit {stock['potential_profit']:.2f}%, Risk {stock['risk'].capitalize()}\n"
            )
            advice += alternative

        # Closing summary
        advice += (
            "\n**Summary**: Based on your goal of a {profit_percentage}% profit, these stocks may align well "
            "with your target. Consider balancing your portfolio with stocks offering a mix of risk levels, "
            "or choose one that best matches your risk tolerance."
        )

    else:
        # Tokenize input text and convert to tensors
        input_text = f"###Human: {user_query} ### Assistant: "
        inputs = tokenizer(input_text, return_tensors="pt").to(device)

        # Ensure that input_ids remain as Long type
        with torch.no_grad():  # Disable gradient computation for inference
            outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=512)

        # Decode and process the model's response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        advice = f"Sorry, I couldn't find any stocks that would match a {profit_percentage}% profit target right now."

    return response + "\n\n" + advice

# Helper function to provide risk explanations based on risk level
def provide_risk_explanation(risk):
    if risk == "high":
        return "higher volatility and less predictability, meaning it might fluctuate significantly."
    elif risk == "medium":
        return "moderate volatility, and while it's less risky, there's still a chance of price variation."
    else:
        return "relatively stable performance, making it a safer option but with potentially slower growth."

# Gradio UI for chatbot
def chatbot_interface(user_query):
    advice = generate_trading_advice(user_query)
    return advice

# Define Gradio app
iface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="Wealth Wiser")

# Launch the Gradio app
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6decf4ca248c963264.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import requests
from bs4 import BeautifulSoup
import uuid
import chromadb
import re
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Initialize ChromaDB client and create/get the collection
client = chromadb.PersistentClient(path="vectorstore")
collection = client.get_or_create_collection(name="portfolio")

# Load your fine-tuned LLaMA 2 model and tokenizer from Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make sure the correct path is added based on your Colab setup (e.g., "/content/model_path")
#tokenizer = AutoTokenizer.from_pretrained("/content/finetuned_llama2")
#model = AutoModelForCausalLM.from_pretrained("/content/finetuned_llama2", torch_dtype=torch.float16).to(device)
model.eval().to(device)  # Ensure model is on the correct device
model.half()

# Function to fetch and process the data (same as in your current scraper)
def fetch_data():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    r = requests.get('https://www.5paisa.com/share-market-today/stocks-to-buy-or-sell-today', headers=headers)

    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('table', {'id': 'stock-table'})

        headers = [header.text.strip() for header in table.find_all('th')]
        rows = []
        for row in table.find_all('tr')[1:]:
            cols = [col.text.strip() for col in row.find_all('td')]
            rows.append(cols)

        data = []
        for row in rows:
            stock_info = dict(zip(headers, row))
            stock_info["TRADE PRICE"] = convert_to_type(stock_info.get("TRADE PRICE", "0"))
            stock_info["SL"] = convert_to_type(stock_info.get("SL", "0"))
            stock_info["TARGET 1"] = convert_to_type(stock_info.get("TARGET 1", "0"))
            stock_info["TARGET 2"] = convert_to_type(stock_info.get("TARGET 2", "0"))
            trade_price = stock_info["TRADE PRICE"]
            target_prices = [stock_info.get("TARGET 1", 0), stock_info.get("TARGET 2", 0)]
            risk_level = calculate_risk(trade_price, target_prices)
            stock_info["RISK"] = risk_level
            data.append(stock_info)

        return data
    else:
        print(f"Failed to fetch page. Status code: {r.status_code}")
        return []

# Define the risk category function
def calculate_risk(trade_price, target_prices):
    trade_price = float(trade_price)
    risk_levels = []

    for target_price in target_prices:
        target_price = float(target_price)
        percentage_difference = ((target_price - trade_price) / trade_price) * 100

        if percentage_difference < 5:
            risk_levels.append('low')
        elif 5 <= percentage_difference < 10:
            risk_levels.append('mid')
        else:
            risk_levels.append('high')

    return max(risk_levels)

# Function to convert string to the appropriate type
def convert_to_type(value):
    try:
        if '.' in value:
            return float(value)
        return int(value)
    except ValueError:
        return value

# Insert or update data (same as in your current logic)
def insert_update_data(data):
    for entry in data:
        stock_name = entry["STOCK"]
        if delete_existing_stock(stock_name):
            print(f"Existing stock '{stock_name}' found and deleted. Updating with new values...")

        collection.add(
            documents=[stock_name],
            metadatas={
                "action": entry["ACTION"],
                "trade_price": entry["TRADE PRICE"],
                "sl": entry["SL"],
                "target_1": entry["TARGET 1"],
                "target_2": entry["TARGET 2"],
                "risk": entry["RISK"]
            },
            ids=[str(uuid.uuid4())]
        )

# Delete existing stock data
def delete_existing_stock(stock_name):
    results = collection.get()
    stock_ids_to_delete = []
    for doc_id, doc, meta in zip(results['ids'], results['documents'], results['metadatas']):
        if meta.get('action') == stock_name:
            stock_ids_to_delete.append(doc_id)

    if stock_ids_to_delete:
        collection.delete(ids=stock_ids_to_delete)
        return True
    return False

# Function to generate trading advice
def generate_trading_advice(user_query):
    # Tokenize input text and convert to tensors
    input_text = f"###Human: {user_query} ### Assistant: "
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Ensure that input_ids remain as Long type
    with torch.no_grad():  # Disable gradient computation for inference
        outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=100)

    # Decode and process the model's response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Now, extract profit target from the user's request using regex
    profit_match = re.search(r'(\d+)% profit', user_query)
    if profit_match:
        profit_percentage = int(profit_match.group(1))
    else:
        profit_percentage = 5  # Default to 5% if not provided

    # Retrieve data from vector DB that matches the user's goal
    matching_stocks = []
    results = collection.get()
    for doc, meta in zip(results['documents'], results['metadatas']):
        target_1 = float(meta["target_1"])
        trade_price = float(meta["trade_price"])
        potential_profit = ((target_1 - trade_price) / trade_price) * 100

        if potential_profit >= profit_percentage:
            matching_stocks.append({
                "stock": doc,
                "trade_price": trade_price,
                "target_1": target_1,
                "risk": meta["risk"],
                "potential_profit": potential_profit
            })

    # Construct the detailed response message for the user
    if matching_stocks:
        advice = f"I found some stocks that may help you achieve a {profit_percentage}% profit:\n\n"

        for stock in matching_stocks:
            # Detail breakdown for each stock
            stock_details = (
                f"**Stock: {stock['stock']}**\n"
                f" - Trade Price: {stock['trade_price']}\n"
                f" - Target Price: {stock['target_1']}\n"
                f" - Potential Profit: {stock['potential_profit']:.2f}%\n"
                f" - Risk Level: {stock['risk'].capitalize()}\n\n"
                f"**Analysis**: If you buy {stock['stock']} at {stock['trade_price']}, you could potentially make a {stock['potential_profit']:.2f}% profit when the price reaches {stock['target_1']}. However, the risk is {stock['risk'].capitalize()}, which indicates {provide_risk_explanation(stock['risk'])}.\n\n"
            )
            advice += stock_details

        # Offer alternative choices based on higher or lower risks/profits
        advice += "**Other choices to consider based on your risk tolerance and profit goals:**\n"
        for stock in matching_stocks:
            # Provide some alternative stocks with a bit more or less profit/risk
            alternative = (
                f"- {stock['stock']}: Trade at {stock['trade_price']}, "
                f"Potential Profit {stock['potential_profit']:.2f}%, Risk {stock['risk'].capitalize()}\n"
            )
            advice += alternative

        # Closing summary
        advice += (
            "\n**Summary**: Based on your goal of a {profit_percentage}% profit, these stocks may align well "
            "with your target. Consider balancing your portfolio with stocks offering a mix of risk levels, "
            "or choose one that best matches your risk tolerance."
        )

    else:
        # Tokenize input text and convert to tensors
        input_text = f"###Human: {user_query} ### Assistant: "
        inputs = tokenizer(input_text, return_tensors="pt").to(device)

        # Ensure that input_ids remain as Long type
        with torch.no_grad():  # Disable gradient computation for inference
            outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=512)

        # Decode and process the model's response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        advice = f"Sorry, I couldn't find any stocks that would match a {profit_percentage}% profit target right now."

    return response + "\n\n" + advice

# Helper function to provide risk explanations based on risk level
def provide_risk_explanation(risk):
    if risk == "high":
        return "higher volatility and less predictability, meaning it might fluctuate significantly."
    elif risk == "medium":
        return "moderate volatility, and while it's less risky, there's still a chance of price variation."
    else:
        return "relatively stable performance, making it a safer option but with potentially slower growth."

# Gradio UI for chatbot
def chatbot_interface(user_query):
    advice = generate_trading_advice(user_query)
    return advice

# Define Gradio app
iface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="Wealth Wiser")

# Launch the Gradio app
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe80d00bf582323906.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **DB Checking Code**

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import uuid
import chromadb

# Initialize ChromaDB client and create/get the collection
client = chromadb.PersistentClient(path="vectorstore")
collection = client.get_or_create_collection(name="portfolio")

# Define the risk category function
def calculate_risk(trade_price, target_prices):
    trade_price = float(trade_price)
    risk_levels = []

    for target_price in target_prices:
        target_price = float(target_price)
        percentage_difference = ((target_price - trade_price) / trade_price) * 100

        if percentage_difference < 5:
            risk_levels.append('low')
        elif 5 <= percentage_difference < 10:
            risk_levels.append('mid')
        else:
            risk_levels.append('high')

    # Use the highest risk level from target prices
    return max(risk_levels)

# Function to fetch and process the data
def fetch_data():
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Make the request with headers
    r = requests.get('https://www.5paisa.com/share-market-today/stocks-to-buy-or-sell-today', headers=headers)

    # Check if the request was successful
    if r.status_code == 200:
        soup = BeautifulSoup(r.content, 'html.parser')

        # Locate the table that contains the data you need
        table = soup.find('table', {'id': 'stock-table'})

        # Extract headers from the table
        headers = [header.text.strip() for header in table.find_all('th')]

        # Extract all rows of the table
        rows = []
        for row in table.find_all('tr')[1:]:  # Skipping the header row
            cols = [col.text.strip() for col in row.find_all('td')]
            rows.append(cols)

        # Prepare the data in dictionary format with correct data types
        data = []
        for row in rows:
            stock_info = dict(zip(headers, row))  # Create a dictionary for each row

            # Convert specific columns to the correct data types
            stock_info["TRADE PRICE"] = convert_to_type(stock_info.get("TRADE PRICE", "0"))
            stock_info["SL"] = convert_to_type(stock_info.get("SL", "0"))
            stock_info["TARGET 1"] = convert_to_type(stock_info.get("TARGET 1", "0"))
            stock_info["TARGET 2"] = convert_to_type(stock_info.get("TARGET 2", "0"))

            # Adding risk level to each stock's data
            trade_price = stock_info["TRADE PRICE"]
            target_prices = [stock_info.get("TARGET 1", 0), stock_info.get("TARGET 2", 0)]
            risk_level = calculate_risk(trade_price, target_prices)
            stock_info["RISK"] = risk_level

            # Add the entry to the data list
            data.append(stock_info)

        return data
    else:
        print(f"Failed to fetch page. Status code: {r.status_code}")
        return []

# Helper function to convert string to appropriate data type
def convert_to_type(value):
    try:
        # Convert to float if it contains a decimal
        if '.' in value:
            return float(value)
        # Convert to int if it is a whole number
        return int(value)
    except ValueError:
        # Return the value as a string if conversion fails
        return value

# Define a function to check if a similar document (by stock name) exists and delete it
def delete_existing_stock(stock_name):
    results = collection.get()  # Fetch all documents

    # Iterate over the results to find matching stock names
    stock_ids_to_delete = []
    for doc_id, doc, meta in zip(results['ids'], results['documents'], results['metadatas']):
        if meta.get('action') == stock_name:  # Match stock names exactly
            stock_ids_to_delete.append(doc_id)

    # Delete all matching stocks
    if stock_ids_to_delete:
        collection.delete(ids=stock_ids_to_delete)  # Delete old entry
        return True  # Stock existed and was deleted
    return False  # Stock did not exist

# Fetch and process the data
data = fetch_data()

# Insert or update data
for entry in data:
    stock_name = entry["STOCK"]

    # Check if the stock already exists and delete if found
    if delete_existing_stock(stock_name):
        print(f"Existing stock '{stock_name}' found and deleted. Updating with new values...")

    # Insert the new or updated stock data
    collection.add(
        documents=[stock_name],
        metadatas={
            "action": entry["ACTION"],
            "trade_price": entry["TRADE PRICE"],
            "sl": entry["SL"],
            "target_1": entry["TARGET 1"],
            "target_2": entry["TARGET 2"],
            "risk": entry["RISK"]
        },
        ids=[str(uuid.uuid4())]
    )

# Retrieve stored documents to verify the insertion
results = collection.get()
for doc, meta in zip(results['documents'], results['metadatas']):
    print(f"Stock: {doc}")
    print(f"Metadata: {meta}")
    print("---")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 13.2MiB/s]


Stock: CANFINHOME
Metadata: {'action': 'BUY', 'risk': 'mid', 'sl': 837, 'target_1': 908, 'target_2': 930, 'trade_price': 872}
---
Stock: SUMICHEM
Metadata: {'action': 'BUY', 'risk': 'mid', 'sl': 504, 'target_1': 535, 'target_2': 545, 'trade_price': 518}
---
Stock: WABAG
Metadata: {'action': 'BUY', 'risk': 'mid', 'sl': 1638, 'target_1': 1775, 'target_2': 1820, 'trade_price': 1706}
---
Stock: CAMS
Metadata: {'action': 'BUY', 'risk': 'mid', 'sl': 4395, 'target_1': 4725, 'target_2': 4840, 'trade_price': 4560}
---
Stock: ALKEM FUT
Metadata: {'action': 'SELL', 'risk': 'low', 'sl': 6105, 'target_1': 5630, 'target_2': 5460, 'trade_price': 5869}
---


# **New Code**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the fine-tuned model
model_path = "./finetuned_granite_7b_instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

In [ ]:
def generate_response(question):
    # Tokenize the input question
    inputs = tokenizer(question, return_tensors="pt").to('cuda')

    # Generate the model's response
    outputs = model.generate(
        inputs['input_ids'],
        max_new_tokens=150,  # You can adjust this to control the length of the generated response
        temperature=0.7,  # Controls randomness in the output (lower value for more focused answers)
        top_p=0.9,  # Nucleus sampling parameter
        do_sample=True,  # Enable sampling for more varied outputs
        num_return_sequences=1  # Number of output sequences
    )

    # Decode the generated output to text
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [ ]:
# Example questions for inference
questions = [
    "What is the best way to save for retirement?",
    "How should I invest my money for long-term growth?",
    "What are the tax benefits of investing in mutual funds?"
]

# Generate responses for each question
for question in questions:
    response = generate_response(question)
    print(f"Question: {question}")
    print(f"Response: {response}\n")